In [32]:
#Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc
from dash.dependencies import Input, Output

### preparation of data sets

In [3]:
aed_location = pd.read_pickle('Data/aed_lat_and_lon.txt')

In [4]:
aed_bxl = aed_location[aed_location['province'] == 'Bruxelles-Brussel']
aed_bxl.shape

(1558, 8)

In [5]:
aed_bxl.head()

,id,address,number,postal_code,municipality,province,latitude,longitude
11,86.0,Schumanplein,11.0,1040.0,Brussel,Bruxelles-Brussel,50.842792,4.384351
14,96.0,Graafschap - Jettelaan,2.0,1090.0,Brussel,Bruxelles-Brussel,50.882962,4.335248
18,101.0,Tweestationsstraat,80.0,1070.0,Brussel,Bruxelles-Brussel,50.831942,4.328980
60,315.0,Wetstraat,23.0,1040.0,Brussel,Bruxelles-Brussel,50.845150,4.369893
69,335.0,Chaussée de Haecht,1405.0,1130.0,Bruxelles,Bruxelles-Brussel,50.883952,4.419213


In [6]:
#subset of the data
df_aed = aed_bxl[['latitude', 'longitude']]
df_aed['cluster'] = None
df_aed['type'] = 'Original AED location'

C:\Users\feng\AppData\Local\Temp\ipykernel_17904\3065688672.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aed['cluster'] = None
C:\Users\feng\AppData\Local\Temp\ipykernel_17904\3065688672.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aed['type'] = 'Original AED location'


In [7]:
df_aed.head()

,latitude,longitude,cluster,type
11,50.842792,4.384351,None,Original AED location
14,50.882962,4.335248,None,Original AED location
18,50.831942,4.328980,None,Original AED location
60,50.845150,4.369893,None,Original AED location
69,50.883952,4.419213,None,Original AED location


In [8]:
df_intervention_aed = pd.read_pickle('Data/df_intervention_aed.txt')
df_intervention_aed.shape

(5156, 13)

In [9]:
df_intervention_aed.head()

,postalcode_intervention,latitude_intervention,longitude_intervention,eventtype_firstcall,eventLevel_firstcall,eventtype_trip,eventlevel_trip,waiting_time,level_num,nearest_aed_id,nearest_aed_dist,nearest_aed_time,nearest_aed_num
28,1140.0,50.87798,4.39229,P011 - Chest pain,N1,P011 - Chest pain,N1,9.0,1,5095.0,294.0,224.0,1
44,1020.0,50.89578,4.35817,P011 - Chest pain,N3,P011 - Chest pain,N3,57.0,3,9980.0,463.0,420.0,1
94,1200.0,50.85592,4.42592,P011 - Chest pain,N1,P011 - Chest pain,N1,14.0,1,5873.0,167.0,135.0,2
95,1200.0,50.85592,4.42592,P011 - Chest pain,N1,P011 - Chest pain,N1,38.0,1,5873.0,167.0,135.0,2
110,1180.0,50.80938,4.34743,P011 - Chest pain,N3,P011 - Chest pain,N3,14.0,3,7649.0,468.0,438.0,1


In [10]:
#subset of the data
df_intervention = df_intervention_aed[['latitude_intervention', 'longitude_intervention']]
df_intervention.columns = ['latitude', 'longitude']
df_intervention['cluster'] = None
df_intervention['type'] = 'Interventions'

C:\Users\feng\AppData\Local\Temp\ipykernel_17904\3825272634.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intervention['cluster'] = None
C:\Users\feng\AppData\Local\Temp\ipykernel_17904\3825272634.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_intervention['type'] = 'Interventions'


In [11]:
df_intervention.head()

,latitude,longitude,cluster,type
28,50.87798,4.39229,None,Interventions
44,50.89578,4.35817,None,Interventions
94,50.85592,4.42592,None,Interventions
95,50.85592,4.42592,None,Interventions
110,50.80938,4.34743,None,Interventions


In [12]:
cluster_center = pd.read_pickle('Data/cluster_centers_nearest_nodes_haversine.txt')
cluster_center.shape

(1562, 5)

the id of each cluster (1562 in total), 
the coordinates of each cluster center (optimized), 
the coordinates of each traffic intersection node that
 is the nearest to the corresponding cluster center

In [13]:
cluster_center.head()

,cluster,latitude_intervention,longitude_intervention,nearest_node_latitude,nearest_node_longitude
0,0,50.835390,4.316295,50.835464,4.316359
1,1,50.862820,4.390080,50.863118,4.389426
2,2,50.814988,4.368908,50.814487,4.368651
3,3,50.888980,4.317545,50.889732,4.317982
4,4,50.815910,4.436210,50.816598,4.436833


In [14]:
#subset of the data
df_optimal_aed = cluster_center[['latitude_intervention', 'longitude_intervention','cluster']]
df_optimal_aed.columns = ['latitude', 'longitude','cluster']
df_optimal_aed['type'] = 'Optimized AED location'

C:\Users\feng\AppData\Local\Temp\ipykernel_17904\1769354971.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_optimal_aed['type'] = 'Optimized AED location'


In [15]:
# subset of the data
df_traffic_intersection = cluster_center[['nearest_node_latitude', 'nearest_node_longitude']]
df_traffic_intersection.columns = ['latitude', 'longitude']
df_traffic_intersection['cluster'] = None
df_traffic_intersection['type'] = 'Traffic intersection'

C:\Users\feng\AppData\Local\Temp\ipykernel_17904\1462128653.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic_intersection['cluster'] = None
C:\Users\feng\AppData\Local\Temp\ipykernel_17904\1462128653.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_traffic_intersection['type'] = 'Traffic intersection'


In [16]:
cluster_all_nodes = pd.read_pickle('Data/df_clusters_all_nodes_haversine.txt')
cluster_all_nodes.shape

(5156, 9)

the coordinates of each intervention, the id of the corresponding cluster, the distance to the nearest cluster center (optimized), the coordinates of the nearest cluster center (optimized), the distance to the corresponding traffic intersection node and its coordinates

In [17]:
cluster_all_nodes.head()

,latitude_intervention,longitude_intervention,cluster,distance_to_aed,aed_latitude,aed_longitude,distance_to_node,node_latitude,node_longitude
28,50.87798,4.39229,402,73,50.877323,4.392230,91,50.877253,4.391667
44,50.89578,4.35817,1323,59,50.895550,4.358940,95,50.895746,4.359525
94,50.85592,4.42592,1036,44,50.855825,4.425305,23,50.855874,4.425594
95,50.85592,4.42592,1036,44,50.855825,4.425305,23,50.855874,4.425594
110,50.80938,4.34743,1499,0,50.809380,4.347430,31,50.809143,4.347664


1. aed_bxl: all orignal aed_location
2. cluster_center: all optimized cluster center(new aed_location) and all nearst traffic intersection location
3. cluster_all_nodes: all intervetnions locations

In [48]:
df = pd.concat([ df_intervention,df_aed, df_optimal_aed, df_traffic_intersection])

In [49]:
df.head()

,latitude,longitude,cluster,type
28,50.87798,4.39229,None,Interventions
44,50.89578,4.35817,None,Interventions
94,50.85592,4.42592,None,Interventions
95,50.85592,4.42592,None,Interventions
110,50.80938,4.34743,None,Interventions


### Visulization

In [ ]:
#create a circle for aed location
coverage_radius = 200

In [50]:
# Create the map using Plotly Express
fig = px.scatter_mapbox(df,
                        lat='latitude',
                        lon='longitude',
                        hover_name='cluster',  # Customize as needed
                        color='type',  # Customize as needed
                        zoom=7,
                        height=600,
                        mapbox_style='open-street-map')
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

In [45]:
# Initialize the Dash app
app = Dash(__name__)

In [51]:
# Define the layout of the app
app.layout = html.Div(children=[
    html.H1(children='Team: Heart Attack'),  # Header
    html.Div(children='''Optimized aed locations with better coverage '''),  # Paragraph
    
    # Embedding the Plotly figure
    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

Click this website after running it: http://127.0.0.1:8050/

In [52]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

##### (folded)

plotly may not able to make fixed cicle around a point

In [33]:
# Function to calculate offset in degrees for a given distance in meters
def calculate_offset(lat, meters):
    # Earth's radius in meters
    R = 6378137
    # Coordinate offsets in radians
    dLat = meters/R
    dLon = meters/(R*np.cos(np.pi*lat/180))
    # Offset in degrees
    lat_offset = dLat * 180/np.pi
    lon_offset = dLon * 180/np.pi
    return lat_offset, lon_offset

In [34]:
# Function to generate points for a circle
def generate_circle(lat, lon, radius):
    lat_offset, lon_offset = calculate_offset(lat, radius)
    return [{
        'type': 'circle',
        'xref': 'x',
        'yref': 'y',
        'x0': lon - lon_offset,
        'y0': lat - lat_offset,
        'x1': lon + lon_offset,
        'y1': lat + lat_offset,
        'line': {
            'color': 'RoyalBlue',
        },
        'fillcolor': 'LightSkyBlue',
        'opacity': 0.5,
    }]

In [38]:
# Sample AED locations
aed_locations = pd.DataFrame({
    'latitude': [40.748817, 40.753817],
    'longitude': [-73.985428, -73.982428],
    'cluster': ['AED 1', 'AED 2'],
})

# Initialize figure
fig = go.Figure()

# Add AED location markers
for index, row in aed_locations.iterrows():
    fig.add_trace(go.Scattermapbox(
        lat=[row['latitude']],
        lon=[row['longitude']],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=20,  # Adjust size to simulate circle radius
            color='rgba(255, 0, 0, 0.5)',  # Semi-transparent 
            opacity=0.5,
        ),
        name=row['cluster'],
    ))

# Set mapbox style and adjust layout
fig.update_layout(
    mapbox=dict(
        style='open-street-map',
        zoom=14,  # Zoom level
        center=dict(lat=aed_locations['latitude'].mean(), lon=aed_locations['longitude'].mean()),  # Map center
    ),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)

fig.show()